In [1]:
# Packages
import os
import yfinance as yf
from haystack.nodes import LinkContentFetcher
from haystack.document_stores import InMemoryDocumentStore, FAISSDocumentStore
from haystack.nodes import  BM25Retriever, AnswerParser, DensePassageRetriever, EmbeddingRetriever
from haystack.nodes.prompt import PromptTemplate
from haystack.nodes import PromptNode
from haystack import Pipeline
from haystack.utils import print_answers

In [2]:
# Model configurations
hfAPIKey = os.getenv("HF_API_KEY")
hfModelName = os.getenv("HF_MODEL_NAME")

In [3]:
gold = yf.Ticker("GC=F")
news = gold.news

In [ ]:
links = []

for new in news:
    links.append(new["link"])

In [ ]:
lcf = LinkContentFetcher()
docs = []

for link in links:
    docs.append(lcf.fetch(link)[0])

In [ ]:
documentStore = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)


documentStore.delete_documents()
documentStore.write_documents(docs)

retriever = DensePassageRetriever(
                document_store=documentStore,
                query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                use_gpu=True,
                embed_title=True,
    )


documentStore.update_embeddings(retriever)

documentStore.save("../../../data/faiss/my_faiss")


In [ ]:
documentStore = FAISSDocumentStore.load(index_path="../../../data/faiss/my_faiss")
documents = documentStore.get_all_documents(return_embedding=True)
print(documents[0].embedding)

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents
relevantDocuments = DocumentSearchPipeline(retriever)
res = relevantDocuments.run(query="Tell me something Gold's market performance", params={"Retriever": {"top_k": 3}})
print_documents(res, max_text_len=512)

In [ ]:
promptTemplate = PromptTemplate(
    prompt="""" You are an assistant getting latest news on the commodity of gold from documents.
                                Strictly answer the following query briefly based on the provided news from the documents and nothing else.
                                If the context does not include an answer, reply with 'The data does not contain information related to the question'.\n
                                Query: {query}\n
                                Documents: {documents}\n
                                Answer: 
                                """,
    output_parser=AnswerParser(),
)

# Prompt Node initialization
promptNode = PromptNode(hfModelName,api_key=hfAPIKey,default_prompt_template = promptTemplate)

In [ ]:
# Pipeline
pipeline = Pipeline()
pipeline.add_node(component=retriever, name="retriever", inputs=["Query"])
pipeline.add_node(component=promptNode, name="promptNode", inputs=["retriever"])

In [ ]:
output = pipeline.run(query="What is the news on King Charles on gold mining",documents=res)
print_answers(output, details="minimum")